In [17]:
import pandas as pd
import duckdb

In [18]:
data1 = pd.read_parquet('/Users/agusvaser/Documents/ITBA/2023/2ndo cuatri/Visualización de la Información/Trabajo Practico/ResultadosElectorales_1v.parquet')
data2 = pd.read_parquet('/Users/agusvaser/Documents/ITBA/2023/2ndo cuatri/Visualización de la Información/Trabajo Practico/ResultadosElectorales_PASO_2023 (1).parquet')

In [19]:
data = duckdb.sql("""select * 
from data1 

union 

select *
from data2 
""")

In [20]:
elecciones_presidenciales = duckdb.sql("""select *
from data
where cargo_id = 1
""")

In [21]:
participacion = duckdb.sql("""select eleccion_tipo, agrupacion_nombre,sum(votos_cantidad) as count 
from elecciones_presidenciales
group by 1,2""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [23]:
agrupaciones = duckdb.sql("""select distinct agrupacion_nombre
from elecciones_presidenciales
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
import altair as alt

colorlist=[]

alt.Chart(participacion).mark_bar().encode(
    x="count",
    y='eleccion_tipo',
    # color=alt.Color('agrupacion_nombre').scale(domain=agrupaciones, range=colorlist)
)


alt.Chart(...)